<a href="https://colab.research.google.com/github/guirms/delta-lake-apache-iceberg/blob/main/ApacheSpark-DeltaLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
!pip install delta-spark

In [ ]:
import pyspark
from delta import *

#  Crie uma sessão Spark com Delta
builder = pyspark.sql.SparkSession.builder.appName("DeltaTutorial") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Obtém uma serssão spark ou cria uma nova
spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Configuração para exibir apenas Logs de erro ou superior.
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
# Carregando dados do data frame Flights 1m.parquet
df = spark.read.parquet("/content/Flights 1m.parquet")

# Inserindo dados na tabela delta
df.write.mode(saveMode="overwrite").format("delta").save("data/delta-table")

In [ ]:
# Lendo os dados
print("Reading delta file ...!")

got_df = spark.read.format("delta").load("data/delta-table")
got_df.show()

In [ ]:
# Configurando para fazer os comandos INSERT, UPDATE E DELETE
from delta.tables import *
from pyspark.sql.types import DateType, ShortType, FloatType

delta_table = DeltaTable.forPath(spark, "/content/data/delta-table")

In [36]:
# Lendo os dados
got_df = spark.read.format("delta").load("data/delta-table")
orderby_df = got_df.orderBy(got_df.FL_DATE.desc())

orderby_df.show()

+----------+---------+---------+--------+--------+---------+---------+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+----------+---------+---------+--------+--------+---------+---------+
|2006-02-28|        4|        7|      30|     183| 8.283334| 9.066667|
|2006-02-28|       -2|      -10|     338|    2475|10.966666|     13.9|
|2006-02-28|      164|      142|     250|    2329|15.983334|    23.45|
|2006-02-28|       -4|       21|     568|    4244|     8.85|    14.75|
|2006-02-28|        1|       -7|      68|     539|     9.55|10.883333|
|2006-02-28|       -7|      -21|     281|    2586|15.883333|23.933332|
|2006-02-28|       -1|        3|      85|     549| 5.983333|      7.6|
|2006-02-28|       -7|        0|     340|    2475| 8.883333|12.166667|
|2006-02-28|       -7|      -15|      85|     626|11.633333|13.216666|
|2006-02-28|       -1|        2|     338|    2475|17.983334|     21.4|
|2006-02-28|       -7|      -16|      84|     626|    13.95|15.466666|
|2006-

In [38]:
# INSERT
from datetime import datetime

data = [(datetime(2024, 4, 25), 12, 10, 103, 650, 17.8, 20.57845)]

schema = StructType([
    StructField("FL_DATE", DateType(), True),
    StructField("DEP_DELAY", ShortType(), True),
    StructField("ARR_DELAY", ShortType(), True),
    StructField("AIR_TIME", ShortType(), True),
    StructField("DISTANCE", ShortType(), True),
    StructField("DEP_TIME", FloatType(), True),
    StructField("ARR_TIME", FloatType(), True),
])

new_df = spark.createDataFrame(data=data, schema=schema)
new_df.write.mode(saveMode="append").format("delta").save("data/delta-table")

In [39]:
# Vendo o novo dado
got_df = spark.read.format("delta").load("data/delta-table")
orderby_df = got_df.orderBy(got_df.FL_DATE.desc())

orderby_df.show()

+----------+---------+---------+--------+--------+---------+---------+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+----------+---------+---------+--------+--------+---------+---------+
|2024-04-25|       12|       10|     103|     650|     17.8| 20.57845|
|2006-02-28|       -2|      -10|     338|    2475|10.966666|     13.9|
|2006-02-28|        4|        7|      30|     183| 8.283334| 9.066667|
|2006-02-28|       -4|       21|     568|    4244|     8.85|    14.75|
|2006-02-28|      164|      142|     250|    2329|15.983334|    23.45|
|2006-02-28|       -7|      -21|     281|    2586|15.883333|23.933332|
|2006-02-28|        1|       -7|      68|     539|     9.55|10.883333|
|2006-02-28|       -7|        0|     340|    2475| 8.883333|12.166667|
|2006-02-28|       -1|        3|      85|     549| 5.983333|      7.6|
|2006-02-28|       -1|        2|     338|    2475|17.983334|     21.4|
|2006-02-28|       -7|      -15|      85|     626|11.633333|13.216666|
|2006-

In [40]:
# UPDATE
delta_table.update(
    condition="FL_DATE = '2024-04-25' AND ARR_DELAY = '10'",
    set={"ARR_DELAY": '12'}
)

In [41]:
# Vendo o dado atualizado
got_df = spark.read.format("delta").load("data/delta-table")
orderby_df = got_df.orderBy(got_df.FL_DATE.desc())

orderby_df.show()

+----------+---------+---------+--------+--------+---------+---------+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+----------+---------+---------+--------+--------+---------+---------+
|2024-04-25|       12|       12|     103|     650|     17.8| 20.57845|
|2006-02-28|       -2|      -10|     338|    2475|10.966666|     13.9|
|2006-02-28|        4|        7|      30|     183| 8.283334| 9.066667|
|2006-02-28|       -4|       21|     568|    4244|     8.85|    14.75|
|2006-02-28|      164|      142|     250|    2329|15.983334|    23.45|
|2006-02-28|       -7|      -21|     281|    2586|15.883333|23.933332|
|2006-02-28|        1|       -7|      68|     539|     9.55|10.883333|
|2006-02-28|       -7|        0|     340|    2475| 8.883333|12.166667|
|2006-02-28|       -1|        3|      85|     549| 5.983333|      7.6|
|2006-02-28|       -1|        2|     338|    2475|17.983334|     21.4|
|2006-02-28|       -7|      -15|      85|     626|11.633333|13.216666|
|2006-

In [42]:
# DELETE
delta_table.delete("FL_DATE = '2024-04-25'")

In [43]:
# Vendo o exclusão do dado
got_df = spark.read.format("delta").load("data/delta-table")
orderby_df = got_df.orderBy(got_df.FL_DATE.desc())

orderby_df.show()

+----------+---------+---------+--------+--------+---------+---------+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+----------+---------+---------+--------+--------+---------+---------+
|2006-02-28|        4|        7|      30|     183| 8.283334| 9.066667|
|2006-02-28|       -2|      -10|     338|    2475|10.966666|     13.9|
|2006-02-28|      164|      142|     250|    2329|15.983334|    23.45|
|2006-02-28|       -4|       21|     568|    4244|     8.85|    14.75|
|2006-02-28|        1|       -7|      68|     539|     9.55|10.883333|
|2006-02-28|       -7|      -21|     281|    2586|15.883333|23.933332|
|2006-02-28|       -1|        3|      85|     549| 5.983333|      7.6|
|2006-02-28|       -7|        0|     340|    2475| 8.883333|12.166667|
|2006-02-28|       -7|      -15|      85|     626|11.633333|13.216666|
|2006-02-28|       -1|        2|     338|    2475|17.983334|     21.4|
|2006-02-28|       -7|      -16|      84|     626|    13.95|15.466666|
|2006-